# Aligment Module

This module guarantees a consistent column ordering between the raw and the competition dataset. This is done such that submissions that are column\-order dependent will not break. Missing values in object type fields are also explicitly filled with a symbolic token, as was done in the competition dataset, for consistency.

In [1]:
import pandas as pd
from collections import OrderedDict
import pprint


class DataMap:
    def __init__(
        self,         
        raw_hhold_file, 
        competition_hhold_file, 
        raw_indiv_file, 
        competition_indiv_file, 
        hhold_map_file, 
        indiv_map_file
    ):
        self.hhold_map = pd.read_json(hhold_map_file, typ='ser')
        self.indiv_map = pd.read_json(indiv_map_file, typ='ser')

        self.raw_hhold_train = pd.read_csv(raw_hhold_file, index_col='id', low_memory=False)
        self.raw_indiv_train = pd.read_csv(raw_indiv_file, index_col=['id', 'iid'], low_memory=False)

        self.raw_hhold_test = pd.read_csv(raw_hhold_file.replace('train.csv', 'test.csv'), index_col='id', low_memory=False)
        self.raw_indiv_test = pd.read_csv(raw_indiv_file.replace('train.csv', 'test.csv'), index_col=['id', 'iid'], low_memory=False)

        self.raw_hhold_train['country'] = 'mwi'
        self.raw_indiv_train['country'] = 'mwi'
        self.raw_hhold_test['country'] = 'mwi'
        self.raw_indiv_test['country'] = 'mwi'
        
        self.competition_hhold_train = pd.read_csv(competition_hhold_file, index_col='id')
        self.competition_indiv_train = pd.read_csv(competition_indiv_file, index_col=['id', 'iid'])

        self.competition_hhold_test = pd.read_csv(competition_hhold_file.replace('train.csv', 'test.csv'), index_col='id')
        self.competition_indiv_test = pd.read_csv(competition_indiv_file.replace('train.csv', 'test.csv'), index_col=['id', 'iid'])
        
        self.hhold_ordered_cols = self.get_competition_ordered_columns(self.raw_hhold_train, self.competition_hhold_train, self.hhold_map)
        self.indiv_ordered_cols = self.get_competition_ordered_columns(self.raw_indiv_train, self.competition_indiv_train, self.indiv_map)
        
        self.raw_indiv_train = self.raw_indiv_train[self.get_ordered_raw_cols(self.indiv_ordered_cols)]
        self.raw_hhold_train = self.raw_hhold_train[self.get_ordered_raw_cols(self.hhold_ordered_cols)]

        self.raw_indiv_test = self.raw_indiv_test[[c for c in self.get_ordered_raw_cols(self.indiv_ordered_cols) if c != 'poor']]
        self.raw_hhold_test = self.raw_hhold_test[[c for c in self.get_ordered_raw_cols(self.hhold_ordered_cols) if c != 'poor']]
    
        self.indiv_vals_map = {}
        self.impute_null_objects()
        self.get_indiv_vals_map()
        
    def get_indiv_vals_map(self):
        for col in self.competition_indiv_train.select_dtypes('object').columns:
            if col == 'country':
                continue

            comp_vals = self.competition_indiv_train[col].value_counts().index
            raw_vals = self.raw_indiv_train[self.indiv_map[col]].value_counts().index

            for i, j in zip(comp_vals, raw_vals):
                if (i not in self.indiv_vals_map):
                    self.indiv_vals_map[i] = f'{self.indiv_map[col]}__{j}'
                else:
                    print(f'Indiv values mapping error on: {col}, {i}, {j}')

    def impute_null_objects(self):
        hhold_empty_map = {c: f'hhold_{ix}_EMPTY' for ix, c in enumerate(self.raw_hhold_train.columns[(self.raw_hhold_train.dtypes == 'O')])}
        indiv_empty_map = {c: f'indiv_{ix}_EMPTY' for ix, c in enumerate(self.raw_indiv_train.columns[(self.raw_indiv_train.dtypes == 'O')])}

        self.raw_hhold_train.fillna(hhold_empty_map, inplace=True)
        self.raw_hhold_test.fillna(hhold_empty_map, inplace=True)
        
        self.raw_indiv_train.fillna(indiv_empty_map, inplace=True)
        self.raw_indiv_test.fillna(indiv_empty_map, inplace=True)

    def get_competition_ordered_columns(self, raw, competition, dmap):
        competition_columns = competition.columns
        
        ordered_cols_map = [{x: x}
            if (
                x == 'poor' or
                x == 'country'
            ) else {dmap.get(x): x} for x in competition_columns
        ]

        return ordered_cols_map

    def get_ordered_raw_cols(self, ordered_cols_map):
        return [k for i in ordered_cols_map for k, v in i.items()]
    
    def get_ordered_competition_cols(self, ordered_cols_map):
        return [v for i in ordered_cols_map for k, v in i.items()]

    def get_feature_mapping(self, competition_features):
        # This method assumes that the first value in an `_` separated
        # column name corresponds to a mappable name in the 
        # column or value of the raw data.
        
        mapped_competition_features = []    

        for col in competition_features:
            cols = col.split('_')
            col = cols[0]
            if col in self.hhold_map:
                col = self.hhold_map[col]

            elif col in self.indiv_map:
                col = self.indiv_map[col]

            elif col in self.indiv_vals_map:
                col = self.indiv_vals_map[col]

            else:
                col = col

            cols[0] = col
            mapped_competition_features.append('_'.join(cols))

        return mapped_competition_features
    
    

In [2]:
data_map = DataMap(
    raw_hhold_file='mwi_hhold_train.csv',
    competition_hhold_file='A_hhold_train.csv',
    raw_indiv_file='mwi_indiv_train.csv',
    competition_indiv_file='A_indiv_train.csv',
    hhold_map_file='mwi_hhold_map.json',
    indiv_map_file='mwi_indiv_map.json'
)

In [3]:
data_map.raw_hhold_train.to_csv('mwi_aligned_hhold_train.csv')
data_map.raw_hhold_test.to_csv('mwi_aligned_hhold_test.csv')

data_map.raw_indiv_train.to_csv('mwi_aligned_indiv_train.csv')
data_map.raw_indiv_test.to_csv('mwi_aligned_indiv_test.csv')

In [4]:
ls -lh

total 108M
-rw-rw-r-- 1 avsolatorio avsolatorio 8.0M Jul 23 21:40 A_hhold_test.csv
-rw-rw-r-- 1 avsolatorio avsolatorio  17M Jul 23 21:40 A_hhold_train.csv
-rw-rw-r-- 1 avsolatorio avsolatorio 4.4M Jul 23 21:40 A_indiv_test.csv
-rw-rw-r-- 1 avsolatorio avsolatorio 9.0M Jul 23 21:40 A_indiv_train.csv
-rw-rw-r-- 1 avsolatorio avsolatorio 6.6M Jul 25 17:52 mwi_aligned_hhold_test.csv
-rw-rw-r-- 1 avsolatorio avsolatorio  14M Jul 25 17:52 mwi_aligned_hhold_train.csv
-rw-rw-r-- 1 avsolatorio avsolatorio 7.8M Jul 25 17:52 mwi_aligned_indiv_test.csv
-rw-rw-r-- 1 avsolatorio avsolatorio  16M Jul 25 17:52 mwi_aligned_indiv_train.csv
-rw-rw-r-- 1 avsolatorio avsolatorio 8.5K Jul 20 13:03 mwi_hhold_map.json
-rw-rw-r-- 1 avsolatorio avsolatorio  15K Jul 20 13:03 mwi_hhold_questions.json
-rw-rw-r-- 1 avsolatorio avsolatorio 5.6M Jul 20 13:03 mwi_hhold_test.csv
-rw-rw-r-- 1 avsolatorio avsolatorio  12M Jul 20 13:03 mwi_hhold_train.csv
-rw-rw-r-- 1 avsolatorio avsolatorio 1.1K Jul 20 13:03 mwi_indiv_m

# Mappings for 1st place winner solution

In [5]:
Model_Keras_Level1_F09M03_clean_mwi_features = ['SlDKnCuu','maLAYXwi', 'vwpsXRGk', 'TYhoEiNm', 'zFkComtB', 'zzwlWZZC', 'DxLvCGgv', 
                 'CbABToOI', 'uSKnVaKV', 'nzTeWUeM', 'nEsgxvAq', 'NmAVTtfA', 'YTdCRVJt', 'QyBloWXZ', 
                 'HKMQJANN', 'ZRrposmO', 'HfKRIwMb', 'UCAmikjV', 'uJYGhXqG', 'bxKGlBYX', 'nCzVgxgY', 
                 'ltcNxFzI', 'MxOgekdE', 'JwtIxvKg', 'bEPKkJXP', 'cqUmYeAp', 'sFWbFEso', 'TqrXZaOw', 
                 'galsfNtg', 'VIRwrkXp', 'gwhBRami', 'bPOwgKnT', 'fpHOwfAs', 'VXXLUaXP', 'btgWptTG', 
                 'YWwNfVtR', 'bgoWYRMQ', 'bMudmjzJ', 'OMtioXZZ', 'bIBQTaHw', 'KcArMKAe', 'wwfmpuWA', 
                 'znHDEHZP', 'kWFVfHWP', 'XwVALSPR', 'HHAeIHna', 'dCGNTMiG', 'ngwuvaCV', 'XSgHIFXD', 
                 'ANBCxZzU', 'NanLCXEI', 'ZnBLVaqz', 'srPNUgVy', 'pCgBHqsR', 'wEbmsuJO', 'pWyRKfsb',
                 'udzhtHIr', 'IZFarbPw', 'lnfulcWk', 'QNLOXNwj', 'YFMZwKrU', 'RJQbcmKy', 'uizuNzbk', 
                 'dlyiMEQt', 'TnWhKowI', 'LoYIbglA', 'GhJKwVWC', 'lVHmBCmb', 'qgxmqJKa', 'gfurxECf', 
                 'hnrnuMte', 'LrQXqVUj', 'XDDOZFWf', 'QayGNSmS', 'ePtrWTFd', 'tbsBPHFD', 'naDKOzdk', 
                 'xkUFKUoW', 'jVDpuAmP', 'SeZULMCT', 'AtGRGAYi', 'WTFJilSZ', 'NBfffJUe', 'mvgxfsRb', 
                 'UXfyiodk', 'EftwspgZ', 'szowPwNq', 'BfGjiYom', 'iWEFJYkR', 'BCehjxAl', 'nqndbwXP', 
                 'phwExnuQ', 'SzUcfjnr', 'PXtHzrqw', 'CNkSTLvx', 'tHFrzjai', 'MKozKLvT', 'pjHvJhoZ', 
                 'zkbPtFyO', 'xZBEXWPR', 'dyGFeFAg', 'pKPTBZZq', 'bCYWWTxH', 'EQKKRGkR', 'cCsFudxF', 
                 'muIetHMK', 'ishdUooQ', 'ItpCDLDM', 'ptEAnCSs', 'orfSPOJX', 'OKMtkqdQ', 'qTginJts',
                 'JzhdOhzb', 'THDtJuYh', 'jwEuQQve', 'rQAsGegu', 'kLkPtNnh', 'CtHqaXhY', 'FmSlImli', 
                 'TiwRslOh', 'PWShFLnY', 'lFExzVaF', 'IKqsuNvV', 'CqqwKRSn', 'YUExUvhq', 'yaHLJxDD', 
                 'qlZMvcWc', 'dqRtXzav', 'ktBqxSwa', 'GIMIxlmv', 'wKVwRQIp', 'UaXLYMMh', 'bKtkhUWD', 
                 'HhKXJWno', 'tAYCAXge', 'aWlBVrkK', 'cDkXTaWP', 'hnmsRSvN', 'GHmAeUhZ', 'BIofZdtd', 
                 'QZiSWCCB', 'CsGvKKBJ', 'OLpGAaEu', 'JCDeZBXq', 'HGPWuGlV', 'WuwrCsIY', 'AlDbXTlZ',
                 'hhold_size', 'ukWqmeSS', 'ukWqmeSS_max', 'ukWqmeSS_min', 'mOlYV_ind_x', 'msICg_ind_x', 
                 'YXCNt_ind_x', 'HgfUG_ind_x', 'EaHvf_ind_x', 'pdgUV_ind_x', 'xrEKh_ind_x', 'QkRds_ind_x', 
                 'XNPgB_ind_x', 'vvXmD_ind_x', 'KOjYm_ind_x', 'Qydia_ind_x', 'vtkRP_ind_x', 'RPBUw_ind_x', 
                 'QQdHS_ind_x', 'Hikoa_ind_x', 'SlRmt_ind_y', 'TRFeI_ind_y', 'fmdsF_ind_y', 'lBMrM_ind_y', 
                 'tMiQp_ind_y', 'wWIzo_ind_y', 'xnnDH_ind_y', 'CXizI_ind_y', 'DQhEE_ind_y', 'LvUxT_ind_y', 
                 'SSvEP_ind_y', 'YsahA_ind_y', 'lzzev_ind_y', 'ccbZA_ind_y', 'fOUHD_ind_y', 'vkRKJ_ind_y', 
                 'rwCRh_ind_y', 'yomtK_ind_y', 'iWGMu_ind_y', 'EaHvf_ind_y', 'GmSKW_ind_y', 'tymHY_ind_y', 
                 'yhUHu_ind_y', 'pdgUV_ind_y', 'qIbMY_ind_y', 'sDvAm_ind_y', 'bszTA_ind_y', 'veBMo_ind_y', 
                 'SowpV_ind_y', 'OeQKE_ind_y', 'XNPgB_ind_y', 'MxNAc_ind_y', 'SuzRU_ind_y', 'PmhpI_ind_y', 
                 'SjaWF_ind_y', 'TUafC_ind_y', 'bazjA_ind_y', 'dpMMl_ind_y', 'qVwNL_ind_y', 'zTqjB_ind_y', 
                 'BNylo_ind_y', 'CXjLj_ind_y', 'PwkMV_ind_y', 'Qydia_ind_y', 'kVYrO_ind_y', 'VneGw_ind_y', 
                 'rXEFU_ind_y', 'aKoLM_ind_y', 'SWhXf_ind_y', 'UCsCT_ind_y', 'uJdwX_ind_y', 'qmOVd_ind_y', 
                 'yOwsR_ind_y', 'ZIrnY_ind_y', 'dAmhs_ind_y', 'gCSRj_ind_y', 'ESfgE_ind_y', 'okwnE_ind_y', 
                 'OkXob_ind_y', 'dDnIb_ind_y', 'jVHyH_ind_y', 'xUYIC_ind_y']
mapped_Model_Keras_Level1_F09M03_clean_mwi_features = data_map.get_feature_mapping(Model_Keras_Level1_F09M03_clean_mwi_features)
pprint.pprint(mapped_Model_Keras_Level1_F09M03_clean_mwi_features, width=100, compact=True)


['cons_0305', 'farm_621', 'geo_district', 'cons_1336', 'gifts201', 'cons_1314', 'cons_0302',
 'farm_604', 'cons_1104', 'farm_615', 'hld_nbcellpho', 'cons_1404', 'hld_adeqcloth', 'cons_0801',
 'farm_623', 'cons_0602', 'cons_0912', 'cons_0101', 'cons_0504', 'cons_0202', 'cons_0824',
 'com_roadtype', 'cons_1417', 'cons_0403', 'cons_0603', 'hld_dwelloccu', 'cons_0413', 'cons_1405',
 'cons_0113', 'cons_0404', 'cons_0703', 'hld_rubbish', 'cons_0513', 'hld_credit1', 'cons_1103',
 'hld_busin9', 'cons_1304', 'cons_1204', 'hld_rooms', 'inc_102', 'cons_1303', 'cons_1324',
 'cons_0503', 'cons_0303', 'cons_1310', 'hld_adeqfood', 'own_528', 'cons_1308', 'hld_electricity',
 'cons_0606', 'cons_0802', 'own_507', 'farm_624', 'own_502', 'cons_0901', 'com_bank', 'cons_0915',
 'cons_0106', 'cons_0104', 'cons_1326', 'hld_selfscale', 'own_501', 'own_518', 'own_513',
 'cons_1214', 'cons_0109', 'cons_0111', 'cons_0204', 'cons_0501', 'cons_1322', 'cons_1101',
 'cons_0816', 'cons_1108', 'cons_1323', 'cons_1321',

In [6]:
Model_KerasUBag_Level1_F11M03_clean = ['SlDKnCuu', 'jdetlNNF', 'maLAYXwi', 'vwpsXRGk', 'TYhoEiNm', 'zFkComtB', 'zzwlWZZC', 
                 'DxLvCGgv', 'CbABToOI', 'qgMygRvX', 'uSKnVaKV', 'nzTeWUeM', 'nEsgxvAq', 'NmAVTtfA', 
                 'YTdCRVJt', 'QyBloWXZ', 'HKMQJANN', 'ZRrposmO', 'HfKRIwMb', 'NRVuZwXK', 'UCAmikjV', 
                 'UGbBCHRE', 'uJYGhXqG', 'bxKGlBYX', 'nCzVgxgY', 'ltcNxFzI', 'JwtIxvKg', 'bEPKkJXP', 
                 'sFWbFEso', 'fHUZugEd', 'TqrXZaOw', 'galsfNtg', 'VIRwrkXp', 'gwhBRami', 'bPOwgKnT', 
                 'fpHOwfAs', 'VXXLUaXP', 'btgWptTG', 'YWwNfVtR', 'bgoWYRMQ', 'bMudmjzJ', 'GKUhYLAE', 
                 'bIBQTaHw', 'KcArMKAe', 'enTUTSQi', 'wwfmpuWA', 'znHDEHZP', 'kWFVfHWP', 'HHAeIHna', 
                 'dCGNTMiG', 'ngwuvaCV', 'XSgHIFXD', 'ANBCxZzU', 'NanLCXEI', 'SqEqFZsM', 'ZnBLVaqz',
                 'srPNUgVy', 'pCgBHqsR', 'wEbmsuJO', 'udzhtHIr', 'IZFarbPw', 'lnfulcWk', 'QNLOXNwj', 
                 'YFMZwKrU', 'RJQbcmKy', 'dlyiMEQt', 'TnWhKowI', 'GhJKwVWC', 'lVHmBCmb', 'qgxmqJKa', 
                 'gfurxECf', 'hnrnuMte', 'XDDOZFWf', 'QayGNSmS', 'ePtrWTFd', 'tbsBPHFD', 'naDKOzdk', 
                 'DNAfxPzs', 'xkUFKUoW', 'jVDpuAmP', 'SeZULMCT', 'AtGRGAYi', 'WTFJilSZ', 'NBfffJUe', 
                 'UXfyiodk', 'EftwspgZ', 'wKcZtLNv', 'szowPwNq', 'BfGjiYom', 'iWEFJYkR', 'BCehjxAl', 
                 'nqndbwXP', 'phwExnuQ', 'SzUcfjnr', 'PXtHzrqw', 'CNkSTLvx', 'tHFrzjai', 'zkbPtFyO', 
                 'xZBEXWPR', 'dyGFeFAg', 'pKPTBZZq', 'bCYWWTxH', 'EQKKRGkR', 'muIetHMK', 'ishdUooQ', 
                 'ItpCDLDM', 'gOGWzlYC', 'ptEAnCSs', 'HDCjCTRd', 'orfSPOJX', 'OKMtkqdQ', 'qTginJts',
                 'jwEuQQve', 'rQAsGegu', 'kLkPtNnh', 'CtHqaXhY', 'FmSlImli', 'TiwRslOh', 'PWShFLnY', 
                 'lFExzVaF', 'IKqsuNvV', 'CqqwKRSn', 'YUExUvhq', 'yaHLJxDD', 'qlZMvcWc', 'ktBqxSwa', 
                 'GIMIxlmv', 'wKVwRQIp', 'UaXLYMMh', 'bKtkhUWD', 'HhKXJWno', 'tAYCAXge', 'aWlBVrkK', 
                 'cDkXTaWP', 'GHmAeUhZ', 'BIofZdtd', 'QZiSWCCB', 'CsGvKKBJ', 'JCDeZBXq', 'HGPWuGlV', 
                 'nEsgxvAq_div_hhold_size', 'OMtioXZZ_div_hhold_size', 'YFMZwKrU_div_hhold_size', 
                 'TiwRslOh_div_hhold_size', 'hhold_size', 'OdXpbPGJ', 'ukWqmeSS', 'ukWqmeSS_max', 
                 'ukWqmeSS_min', 'kzSFB_ind_x', 'mOlYV_ind_x', 'axSTs_ind_x', 'YXCNt_ind_x', 'oArAw_ind_x', 
                 'scxJu_ind_x', 'VzUws_ind_x', 'YwljV_ind_x', 'QkRds_ind_x', 'nUKzL_ind_x', 'OeQKE_ind_x', 
                 'XNPgB_ind_x', 'dpMMl_ind_x', 'ndArQ_ind_x', 'GIApU_ind_x', 'Qydia_ind_x', 'vtkRP_ind_x',
                 'sitaC_ind_x', 'VneGw_ind_x', 'rXEFU_ind_x', 'EAWFH_ind_x', 'UCsCT_ind_x', 'XQevi_ind_x', 
                 'QQdHS_ind_x', 'uEstx_ind_x', 'Hikoa_ind_x', 'rkLqZ_ind_x', 'FUUXv_ind_x', 'juMSt_ind_x', 
                 'SlRmt_ind_y', 'TRFeI_ind_y', 'dHZCo_ind_y', 'duBym_ind_y', 'lBMrM_ind_y', 'oGavK_ind_y', 
                 'tMiQp_ind_y', 'wWIzo_ind_y', 'xnnDH_ind_y', 'yAyAe_ind_y', 'FRcdT_ind_y', 'UFoKR_ind_y',
                 'CXizI_ind_y', 'JyIRx_ind_y', 'YsahA_ind_y', 'lzzev_ind_y', 'msICg_ind_y', 'NDnCs_ind_y', 
                 'QyhRH_ind_y', 'XvoCa_ind_y', 'ccbZA_ind_y', 'fOUHD_ind_y', 'xMiWa_ind_y', 'bJTYb_ind_y', 
                 'rwCRh_ind_y', 'scxJu_ind_y', 'OMzWB_ind_y', 'DgtXD_ind_y', 'EaHvf_ind_y', 'GmSKW_ind_y', 
                 'VzUws_ind_y', 'uhOlG_ind_y', 'zfTDU_ind_y', 'IZbuU_ind_y', 'olfwp_ind_y', 'pdgUV_ind_y',
                 'qIbMY_ind_y', 'sDvAm_ind_y', 'BQEnF_ind_y', 'Rjkzz_ind_y', 'VGNER_ind_y', 'bszTA_ind_y', 
                 'xBZrP_ind_y', 'veBMo_ind_y', 'SowpV_ind_y', 'nUKzL_ind_y', 'OeQKE_ind_y', 'vSaJn_ind_y', 
                 'CneHb_ind_y', 'JPCna_ind_y', 'MxNAc_ind_y', 'vvXmD_ind_y', 'TUafC_ind_y', 'dpMMl_ind_y', 
                 'ndArQ_ind_y', 'zTqjB_ind_y', 'BNylo_ind_y', 'CXjLj_ind_y', 'AyuSE_ind_y', 'ZApCl_ind_y',
                 'hCKQi_ind_y', 'Qydia_ind_y', 'vtkRP_ind_y', 'kVYrO_ind_y', 'VneGw_ind_y', 'rXEFU_ind_y', 
                 'zncPX_ind_y', 'aKoLM_ind_y', 'DGyQh_ind_y', 'cEcbt_ind_y', 'xjHpn_ind_y', 'QBrMF_ind_y', 
                 'mEGPl_ind_y', 'dAmhs_ind_y', 'gCSRj_ind_y', 'ESfgE_ind_y', 'Coacj_ind_y', 'dDnIb_ind_y', 
                 'jVHyH_ind_y', 'rkLqZ_ind_y', 'xUYIC_ind_y', 'GtHel_ind_y', 'juMSt_ind_y']
mapped_Model_KerasUBag_Level1_F11M03_clean = data_map.get_feature_mapping(Model_KerasUBag_Level1_F11M03_clean)
pprint.pprint(mapped_Model_KerasUBag_Level1_F11M03_clean, width=100, compact=True)




['cons_0305', 'cons_0408', 'farm_621', 'geo_district', 'cons_1336', 'gifts201', 'cons_1314',
 'cons_0302', 'farm_604', 'hld_lighting', 'cons_1104', 'farm_615', 'hld_nbcellpho', 'cons_1404',
 'hld_adeqcloth', 'cons_0801', 'farm_623', 'cons_0602', 'cons_0912', 'hld_walls', 'cons_0101',
 'com_schoolelec', 'cons_0504', 'cons_0202', 'cons_0824', 'com_roadtype', 'cons_0403', 'cons_0603',
 'cons_0413', 'cons_0905', 'cons_1405', 'cons_0113', 'cons_0404', 'cons_0703', 'hld_rubbish',
 'cons_0513', 'hld_credit1', 'cons_1103', 'hld_busin9', 'cons_1304', 'cons_1204', 'farm_616',
 'inc_102', 'cons_1303', 'cons_0911', 'cons_1324', 'cons_0503', 'cons_0303', 'hld_adeqfood',
 'own_528', 'cons_1308', 'hld_electricity', 'cons_0606', 'cons_0802', 'own_510', 'own_507',
 'farm_624', 'own_502', 'cons_0901', 'cons_0915', 'cons_0106', 'cons_0104', 'cons_1326',
 'hld_selfscale', 'own_501', 'own_513', 'cons_1214', 'cons_0111', 'cons_0204', 'cons_0501',
 'cons_1322', 'cons_1101', 'cons_1108', 'cons_1323', 'cons_13

In [7]:
Model_KerasUBag_Level1_F08M03_clean = ['KAJOWiiw', 'DsKacCdL', 'rtPrBBPl', 'tMJrvvut', 'TYhoEiNm',
                    'bgfNZfcj', 'sYIButva', 'VZtBaoXL', 'GUvFHPNA', 'fxbqfEWb',
                    'nGTepfos', 'CbABToOI', 'uSKnVaKV', 'hESBInAl', 'BbKZUYsB',
                    'UCnazcxd', 'aCfsveTu', 'EfkPrfXa', 'FcekeISI', 'wakWLjkG',
                    'dkoIJCbY', 'NrUWfvEq', 'WqhniYIc', 'IIEHQNUc', 'UGbBCHRE',
                    'bxKGlBYX', 'MxOgekdE', 'ggNglVqE', 'YDgWYWcJ', 'SqGRfEuW',
                    'benRXROb', 'dSALvhyd', 'gfmfEyjQ', 'WbxAxHul', 'FlBqizNL',
                    'KjkrfGLD', 'JbjHTYUM', 'HmDAlkAH', 'galsfNtg', 'dsIjcEFe',
                    'OybQOufM', 'ihGjxdDj', 'FGYOIJbC', 'UBanubTh', 'NIRMacrk',
                    'wxDnGIwN', 'rAkSnhJF', 'glEjrMIg', 'GKUhYLAE', 'SsZAZLma',
                    'KcArMKAe', 'TFrimNtw', 'LjvKYNON', 'wwfmpuWA', 'TvShZEBA',
                    'nuwxPLMe', 'eeYoszDM', 'HHAeIHna', 'CrfscGZl', 'SqEqFZsM',
                    'lFcfBRGd', 'AsEmHUzj', 'pyBSpOoN', 'srPNUgVy', 'TWXCrjor',
                    'wgWdGBOp', 'ErggjCIN', 'lnfulcWk', 'UHGnBrNt', 'QNLOXNwj',
                    'ytYMzOlW', 'ucXrHdoC', 'iBQXwnGC', 'sslNoPlw', 'InULRrrv',
                    'LoYIbglA', 'EuJrVjyG', 'nSzbETYS', 'CpqWSQcW', 'XqURHMoh',
                    'mDTcQhdH', 'mvGdZZcs', 'CbzSWtkF', 'LrQXqVUj', 'CIGUXrRQ',
                    'CtFxPQPT', 'ePtrWTFd', 'lTAXSTys', 'dEpQghsA', 'SeZULMCT',
                    'NitzgUzY', 'YlZCqMNw', 'rYvVKPAF', 'rfDBJtIz', 'KHzKOKPw',
                    'EftwspgZ', 'mycoyYwl', 'ySkAFOzx', 'dkPWxwSF', 'bSaLisbO',
                    'wKcZtLNv', 'mBlWbDmc', 'szowPwNq', 'ULMvnWcn', 'ogHwwdzc',
                    'uZGqTQUP', 'PXtHzrqw', 'MKozKLvT', 'zkbPtFyO', 'HfOrdgBo',
                    'YKwvJgoP', 'rnJOTwVD', 'xNUUjCIL', 'JMNvdasy', 'MBQcYnjc',
                    'cCsFudxF', 'hJrMTBVd', 'ishdUooQ', 'gOGWzlYC', 'HDCjCTRd',
                    'lOujHrCk', 'MARfVwUE', 'orfSPOJX', 'QBJeqwPF', 'JzhdOhzb',
                    'THDtJuYh', 'nKoaotpH', 'TzPdCEPV', 'DbUNVFwv', 'UsENDgsH',
                    'PWShFLnY', 'uRFXnNKV', 'CVCsOVew', 'tlxXCDiW', 'CqqwKRSn',
                    'YUExUvhq', 'UXhTXbuS', 'yaHLJxDD', 'zuMWFXax', 'ALbGNqKm',
                    'tOWnWxYe', 'RvTenIlS', 'wKVwRQIp', 'ncjMNgfp', 'RJFKdmYJ',
                    'gllMXToa', 'VFTkSOrq', 'WAFKMNwv', 'mHzqKSuN', 'UjuNwfjv',
                    'cDkXTaWP', 'GHmAeUhZ', 'VBjVVDwp', 'kZVpcgJL', 'sDGibZrP',
                    'OLpGAaEu', 'LrDrWRjC', 'AlDbXTlZ']
mapped_Model_KerasUBag_Level1_F08M03_clean = data_map.get_feature_mapping(Model_KerasUBag_Level1_F08M03_clean)
pprint.pprint(mapped_Model_KerasUBag_Level1_F08M03_clean, width=100, compact=True)


    

['inc_112', 'cons_1205', 'cons_1329', 'cons_0306', 'cons_1336', 'cons_0103', 'cons_0810', 'inc_109',
 'cons_0115', 'cons_1311', 'cons_1309', 'farm_604', 'cons_1104', 'inc_111', 'hld_busin2',
 'com_postoffice', 'cons_0610', 'own_505', 'farm_618', 'own_516', 'cons_0910', 'inc_105',
 'cons_0514', 'cons_1330', 'com_schoolelec', 'cons_0202', 'cons_1417', 'cons_0401', 'own_519',
 'hld_adeqhealth', 'cons_0507', 'cons_1338', 'hld_toiletshr', 'cons_0309', 'cons_0201',
 'com_distclinic', 'farm_622', 'inc_110', 'cons_0113', 'cons_0414', 'cons_0411', 'cons_0827',
 'own_532', 'farm_601', 'cons_1331', 'cons_1325', 'com_distprimary', 'cons_1102', 'farm_616',
 'cons_0608', 'cons_1303', 'cons_1306', 'hld_floor', 'cons_1324', 'cons_0814', 'cons_1315',
 'cons_0409', 'hld_adeqfood', 'hld_bednet', 'own_510', 'own_522', 'hld_headsleep', 'cons_0515',
 'farm_624', 'inc_113', 'own_511', 'cons_1319', 'cons_0104', 'inc_106', 'cons_1326', 'cons_1327',
 'inc_101', 'cons_1105', 'cons_1301', 'cons_1410', 'cons_0109'

In [8]:
Model_LightGBM_Level1_F11M01_clean = ['SlDKnCuu', 'jdetlNNF', 'maLAYXwi', 'vwpsXRGk', 'TYhoEiNm', 'zFkComtB', 'zzwlWZZC', 
                 'DxLvCGgv', 'CbABToOI', 'qgMygRvX', 'uSKnVaKV', 'nzTeWUeM', 'nEsgxvAq', 'NmAVTtfA', 
                 'YTdCRVJt', 'QyBloWXZ', 'HKMQJANN', 'ZRrposmO', 'HfKRIwMb', 'NRVuZwXK', 'UCAmikjV', 
                 'UGbBCHRE', 'uJYGhXqG', 'bxKGlBYX', 'nCzVgxgY', 'ltcNxFzI', 'JwtIxvKg', 'bEPKkJXP', 
                 'sFWbFEso', 'fHUZugEd', 'TqrXZaOw', 'galsfNtg', 'VIRwrkXp', 'gwhBRami', 'bPOwgKnT', 
                 'fpHOwfAs', 'VXXLUaXP', 'btgWptTG', 'YWwNfVtR', 'bgoWYRMQ', 'bMudmjzJ', 'GKUhYLAE', 
                 'bIBQTaHw', 'KcArMKAe', 'enTUTSQi', 'wwfmpuWA', 'znHDEHZP', 'kWFVfHWP', 'HHAeIHna', 
                 'dCGNTMiG', 'ngwuvaCV', 'XSgHIFXD', 'ANBCxZzU', 'NanLCXEI', 'SqEqFZsM', 'ZnBLVaqz',
                 'srPNUgVy', 'pCgBHqsR', 'wEbmsuJO', 'udzhtHIr', 'IZFarbPw', 'lnfulcWk', 'QNLOXNwj', 
                 'YFMZwKrU', 'RJQbcmKy', 'dlyiMEQt', 'TnWhKowI', 'GhJKwVWC', 'lVHmBCmb', 'qgxmqJKa', 
                 'gfurxECf', 'hnrnuMte', 'XDDOZFWf', 'QayGNSmS', 'ePtrWTFd', 'tbsBPHFD', 'naDKOzdk', 
                 'DNAfxPzs', 'xkUFKUoW', 'jVDpuAmP', 'SeZULMCT', 'AtGRGAYi', 'WTFJilSZ', 'NBfffJUe', 
                 'UXfyiodk', 'EftwspgZ', 'wKcZtLNv', 'szowPwNq', 'BfGjiYom', 'iWEFJYkR', 'BCehjxAl', 
                 'nqndbwXP', 'phwExnuQ', 'SzUcfjnr', 'PXtHzrqw', 'CNkSTLvx', 'tHFrzjai', 'zkbPtFyO', 
                 'xZBEXWPR', 'dyGFeFAg', 'pKPTBZZq', 'bCYWWTxH', 'EQKKRGkR', 'muIetHMK', 'ishdUooQ', 
                 'ItpCDLDM', 'gOGWzlYC', 'ptEAnCSs', 'HDCjCTRd', 'orfSPOJX', 'OKMtkqdQ', 'qTginJts',
                 'jwEuQQve', 'rQAsGegu', 'kLkPtNnh', 'CtHqaXhY', 'FmSlImli', 'TiwRslOh', 'PWShFLnY', 
                 'lFExzVaF', 'IKqsuNvV', 'CqqwKRSn', 'YUExUvhq', 'yaHLJxDD', 'qlZMvcWc', 'ktBqxSwa', 
                 'GIMIxlmv', 'wKVwRQIp', 'UaXLYMMh', 'bKtkhUWD', 'HhKXJWno', 'tAYCAXge', 'aWlBVrkK', 
                 'cDkXTaWP', 'GHmAeUhZ', 'BIofZdtd', 'QZiSWCCB', 'CsGvKKBJ', 'JCDeZBXq', 'HGPWuGlV', 
                 'nEsgxvAq_div_hhold_size', 'OMtioXZZ_div_hhold_size', 'YFMZwKrU_div_hhold_size', 
                 'TiwRslOh_div_hhold_size', 'hhold_size', 'OdXpbPGJ', 'ukWqmeSS', 'ukWqmeSS_max', 
                 'ukWqmeSS_min', 'kzSFB_ind_x', 'mOlYV_ind_x', 'axSTs_ind_x', 'YXCNt_ind_x', 'oArAw_ind_x', 
                 'scxJu_ind_x', 'VzUws_ind_x', 'YwljV_ind_x', 'QkRds_ind_x', 'nUKzL_ind_x', 'OeQKE_ind_x', 
                 'XNPgB_ind_x', 'dpMMl_ind_x', 'ndArQ_ind_x', 'GIApU_ind_x', 'Qydia_ind_x', 'vtkRP_ind_x',
                 'sitaC_ind_x', 'VneGw_ind_x', 'rXEFU_ind_x', 'EAWFH_ind_x', 'UCsCT_ind_x', 'XQevi_ind_x', 
                 'QQdHS_ind_x', 'uEstx_ind_x', 'Hikoa_ind_x', 'rkLqZ_ind_x', 'FUUXv_ind_x', 'juMSt_ind_x', 
                 'SlRmt_ind_y', 'TRFeI_ind_y', 'dHZCo_ind_y', 'duBym_ind_y', 'lBMrM_ind_y', 'oGavK_ind_y', 
                 'tMiQp_ind_y', 'wWIzo_ind_y', 'xnnDH_ind_y', 'yAyAe_ind_y', 'FRcdT_ind_y', 'UFoKR_ind_y',
                 'CXizI_ind_y', 'JyIRx_ind_y', 'YsahA_ind_y', 'lzzev_ind_y', 'msICg_ind_y', 'NDnCs_ind_y', 
                 'QyhRH_ind_y', 'XvoCa_ind_y', 'ccbZA_ind_y', 'fOUHD_ind_y', 'xMiWa_ind_y', 'bJTYb_ind_y', 
                 'rwCRh_ind_y', 'scxJu_ind_y', 'OMzWB_ind_y', 'DgtXD_ind_y', 'EaHvf_ind_y', 'GmSKW_ind_y', 
                 'VzUws_ind_y', 'uhOlG_ind_y', 'zfTDU_ind_y', 'IZbuU_ind_y', 'olfwp_ind_y', 'pdgUV_ind_y',
                 'qIbMY_ind_y', 'sDvAm_ind_y', 'BQEnF_ind_y', 'Rjkzz_ind_y', 'VGNER_ind_y', 'bszTA_ind_y', 
                 'xBZrP_ind_y', 'veBMo_ind_y', 'SowpV_ind_y', 'nUKzL_ind_y', 'OeQKE_ind_y', 'vSaJn_ind_y', 
                 'CneHb_ind_y', 'JPCna_ind_y', 'MxNAc_ind_y', 'vvXmD_ind_y', 'TUafC_ind_y', 'dpMMl_ind_y', 
                 'ndArQ_ind_y', 'zTqjB_ind_y', 'BNylo_ind_y', 'CXjLj_ind_y', 'AyuSE_ind_y', 'ZApCl_ind_y',
                 'hCKQi_ind_y', 'Qydia_ind_y', 'vtkRP_ind_y', 'kVYrO_ind_y', 'VneGw_ind_y', 'rXEFU_ind_y', 
                 'zncPX_ind_y', 'aKoLM_ind_y', 'DGyQh_ind_y', 'cEcbt_ind_y', 'xjHpn_ind_y', 'QBrMF_ind_y', 
                 'mEGPl_ind_y', 'dAmhs_ind_y', 'gCSRj_ind_y', 'ESfgE_ind_y', 'Coacj_ind_y', 'dDnIb_ind_y', 
                 'jVHyH_ind_y', 'rkLqZ_ind_y', 'xUYIC_ind_y', 'GtHel_ind_y', 'juMSt_ind_y']    
    
mapped_Model_LightGBM_Level1_F11M01_clean = data_map.get_feature_mapping(Model_LightGBM_Level1_F11M01_clean)
pprint.pprint(mapped_Model_LightGBM_Level1_F11M01_clean, width=100, compact=True)




['cons_0305', 'cons_0408', 'farm_621', 'geo_district', 'cons_1336', 'gifts201', 'cons_1314',
 'cons_0302', 'farm_604', 'hld_lighting', 'cons_1104', 'farm_615', 'hld_nbcellpho', 'cons_1404',
 'hld_adeqcloth', 'cons_0801', 'farm_623', 'cons_0602', 'cons_0912', 'hld_walls', 'cons_0101',
 'com_schoolelec', 'cons_0504', 'cons_0202', 'cons_0824', 'com_roadtype', 'cons_0403', 'cons_0603',
 'cons_0413', 'cons_0905', 'cons_1405', 'cons_0113', 'cons_0404', 'cons_0703', 'hld_rubbish',
 'cons_0513', 'hld_credit1', 'cons_1103', 'hld_busin9', 'cons_1304', 'cons_1204', 'farm_616',
 'inc_102', 'cons_1303', 'cons_0911', 'cons_1324', 'cons_0503', 'cons_0303', 'hld_adeqfood',
 'own_528', 'cons_1308', 'hld_electricity', 'cons_0606', 'cons_0802', 'own_510', 'own_507',
 'farm_624', 'own_502', 'cons_0901', 'cons_0915', 'cons_0106', 'cons_0104', 'cons_1326',
 'hld_selfscale', 'own_501', 'own_513', 'cons_1214', 'cons_0111', 'cons_0204', 'cons_0501',
 'cons_1322', 'cons_1101', 'cons_1108', 'cons_1323', 'cons_13

In [9]:
Model_LightGBM_Level1_F10M01_clean = ['SlDKnCuu', 'jdetlNNF', 'vwpsXRGk', 'TYhoEiNm', 'VZtBaoXL', 'zFkComtB', 'zzwlWZZC', 
                 'DxLvCGgv', 'CbABToOI', 'qgMygRvX', 'uSKnVaKV', 'nEsgxvAq', 'NmAVTtfA', 'YTdCRVJt', 
                 'QyBloWXZ', 'HKMQJANN', 'ZRrposmO', 'EJgrQqET', 'HfKRIwMb', 'NRVuZwXK', 'UCAmikjV', 
                 'UGbBCHRE', 'uJYGhXqG', 'bxKGlBYX', 'nCzVgxgY', 'MxOgekdE', 'SqGRfEuW', 'JwtIxvKg', 
                 'bEPKkJXP', 'cqUmYeAp', 'sFWbFEso', 'TqrXZaOw', 'VIRwrkXp', 'gwhBRami', 'bPOwgKnT', 
                 'fpHOwfAs', 'VXXLUaXP', 'btgWptTG', 'YWwNfVtR', 'bgoWYRMQ', 'bMudmjzJ', 'GKUhYLAE',
                 'OMtioXZZ', 'bIBQTaHw', 'KcArMKAe', 'enTUTSQi', 'wwfmpuWA', 'znHDEHZP', 'kWFVfHWP', 
                 'XwVALSPR', 'CrfscGZl', 'dCGNTMiG', 'ngwuvaCV', 'XSgHIFXD', 'ANBCxZzU', 'NanLCXEI', 
                 'ZnBLVaqz', 'srPNUgVy', 'pCgBHqsR', 'wEbmsuJO', 'TWXCrjor', 'mRgnuJVE', 'pWyRKfsb', 
                 'udzhtHIr', 'IZFarbPw', 'QNLOXNwj', 'YFMZwKrU', 'RJQbcmKy', 'TnWhKowI', 'LoYIbglA', 
                 'GhJKwVWC', 'lVHmBCmb', 'qgxmqJKa', 'gfurxECf', 'hnrnuMte', 'XDDOZFWf', 'ccAHraiP', 
                 'QayGNSmS', 'ePtrWTFd', 'tbsBPHFD', 'naDKOzdk', 'DNAfxPzs', 'xkUFKUoW', 'SeZULMCT', 
                 'AtGRGAYi', 'FGDcbVBN', 'WTFJilSZ', 'NBfffJUe', 'mvgxfsRb', 'UXfyiodk', 'EftwspgZ', 
                 'bSaLisbO', 'wKcZtLNv', 'BfGjiYom', 'iWEFJYkR', 'BCehjxAl', 'CHAQHqqr', 'nqndbwXP',
                 'phwExnuQ', 'SzUcfjnr', 'PXtHzrqw', 'CNkSTLvx', 'MKozKLvT', 'zkbPtFyO', 'xZBEXWPR', 
                 'dyGFeFAg', 'bCYWWTxH', 'EQKKRGkR', 'muIetHMK', 'ItpCDLDM', 'gOGWzlYC', 'ptEAnCSs', 
                 'HDCjCTRd', 'orfSPOJX', 'OKMtkqdQ', 'qTginJts', 'JzhdOhzb', 'jwEuQQve', 'rQAsGegu', 
                 'kLkPtNnh', 'CtHqaXhY', 'FmSlImli', 'TiwRslOh', 'PWShFLnY', 'lFExzVaF', 'IKqsuNvV',
                 'CqqwKRSn', 'YUExUvhq','yaHLJxDD', 'qlZMvcWc', 'dqRtXzav', 'ktBqxSwa', 'NqPjMmKP',
                 'GIMIxlmv', 'UaXLYMMh', 'bKtkhUWD', 'HhKXJWno', 'tAYCAXge', 'WAFKMNwv', 'aWlBVrkK', 
                 'cDkXTaWP', 'hnmsRSvN', 'GHmAeUhZ', 'BIofZdtd', 'QZiSWCCB', 'CsGvKKBJ', 'OLpGAaEu', 
                 'JCDeZBXq', 'WuwrCsIY', 'AlDbXTlZ', 'hhold_size', 'OdXpbPGJ', 'ukWqmeSS', 'ukWqmeSS_max', 
                 'ukWqmeSS_min', 'mOlYV_ind_x', 'JyIRx_ind_x', 'msICg_ind_x', 'YXCNt_ind_x', 'oArAw_ind_x', 
                 'HgfUG_ind_x', 'tqINY_ind_x', 'EaHvf_ind_x', 'GmSKW_ind_x', 'pdgUV_ind_x', 'xrEKh_ind_x', 
                 'QkRds_ind_x', 'TGbFh_ind_x', 'veIDf_ind_x', 'vvXmD_ind_x', 'ndArQ_ind_x', 'KOjYm_ind_x', 
                 'hCKQi_ind_x', 'Qydia_ind_x', 'vtkRP_ind_x', 'EAWFH_ind_x', 'xjHpn_ind_x', 'RPBUw_ind_x', 
                 'yOwsR_ind_x', 'dAmhs_ind_x', 'uEstx_ind_x', 'OkXob_ind_x', 'zQvdC_ind_x', 'juMSt_ind_x', 
                 'JTCKs_ind_x', 'SlRmt_ind_y', 'TRFeI_ind_y', 'dHZCo_ind_y', 'duBym_ind_y', 'oGavK_ind_y', 
                 'tMiQp_ind_y', 'wWIzo_ind_y', 'mOlYV_ind_y', 'CXizI_ind_y', 'DQhEE_ind_y','HIvIU_ind_y', 
                 'JyIRx_ind_y', 'LvUxT_ind_y','YsahA_ind_y', 'AvBOo_ind_y', 'BqqGq_ind_y', 'QyhRH_ind_y', 
                 'ccbZA_ind_y', 'fOUHD_ind_y', 'pWLuE_ind_y', 'kpkiH_ind_y', 'rwCRh_ind_y', 'OMzWB_ind_y',
                 'Whopv_ind_y', 'cHNSE_ind_y', 'zCwHm_ind_y', 'AYcgs_ind_y', 'DgtXD_ind_y', 'EaHvf_ind_y', 
                 'GmSKW_ind_y', 'pRitH_ind_y', 'yhUHu_ind_y', 'zfTDU_ind_y', 'kzJXk_ind_y', 'pdgUV_ind_y', 
                 'qIbMY_ind_y', 'sDvAm_ind_y', 'xrEKh_ind_y', 'bszTA_ind_y', 'xBZrP_ind_y', 'veBMo_ind_y', 
                 'SowpV_ind_y', 'OeQKE_ind_y', 'XNPgB_ind_y', 'veIDf_ind_y', 'MxNAc_ind_y', 'SuzRU_ind_y',
                 'PaHYu_ind_y', 'SjaWF_ind_y', 'TUafC_ind_y', 'dpMMl_ind_y', 'meQRz_ind_y', 'zTqjB_ind_y',
                 'BNylo_ind_y', 'CXjLj_ind_y', 'PwkMV_ind_y', 'GxyHv_ind_y', 'PrZhn_ind_y', 'ZApCl_ind_y',
                 'hCKQi_ind_y', 'Qydia_ind_y', 'vtkRP_ind_y', 'kVYrO_ind_y', 'OoqEw_ind_y', 'SWhXf_ind_y', 
                 'UCsCT_ind_y', 'uJdwX_ind_y', 'QBrMF_ind_y', 'mEGPl_ind_y', 'qmOVd_ind_y', 'yOwsR_ind_y', 
                 'Jarbl_ind_y', 'dAmhs_ind_y', 'ESfgE_ind_y', 'okwnE_ind_y', 'xUYIC_ind_y', 'GtHel_ind_y', 
                 'vhhVz_ind_y']
mapped_Model_LightGBM_Level1_F10M01_clean = data_map.get_feature_mapping(Model_LightGBM_Level1_F10M01_clean)
pprint.pprint(mapped_Model_LightGBM_Level1_F10M01_clean, width=100, compact=True)




['cons_0305', 'cons_0408', 'geo_district', 'cons_1336', 'inc_109', 'gifts201', 'cons_1314',
 'cons_0302', 'farm_604', 'hld_lighting', 'cons_1104', 'hld_nbcellpho', 'cons_1404',
 'hld_adeqcloth', 'cons_0801', 'farm_623', 'cons_0602', 'cons_1220', 'cons_0912', 'hld_walls',
 'cons_0101', 'com_schoolelec', 'cons_0504', 'cons_0202', 'cons_0824', 'cons_1417',
 'hld_adeqhealth', 'cons_0403', 'cons_0603', 'hld_dwelloccu', 'cons_0413', 'cons_1405', 'cons_0404',
 'cons_0703', 'hld_rubbish', 'cons_0513', 'hld_credit1', 'cons_1103', 'hld_busin9', 'cons_1304',
 'cons_1204', 'farm_616', 'hld_rooms', 'inc_102', 'cons_1303', 'cons_0911', 'cons_1324',
 'cons_0503', 'cons_0303', 'cons_1310', 'hld_bednet', 'own_528', 'cons_1308', 'hld_electricity',
 'cons_0606', 'cons_0802', 'own_507', 'farm_624', 'own_502', 'cons_0901', 'inc_113', 'inc_115',
 'com_bank', 'cons_0915', 'cons_0106', 'cons_1326', 'hld_selfscale', 'own_501', 'cons_1214',
 'cons_0109', 'cons_0111', 'cons_0204', 'cons_0501', 'cons_1322', 'cons

In [10]:
Model_LightGBM_Level1_F09M01_clean = ['SlDKnCuu', 'maLAYXwi', 'vwpsXRGk', 'TYhoEiNm', 'zFkComtB', 'zzwlWZZC', 'DxLvCGgv', 'CbABToOI',
                 'uSKnVaKV', 'nzTeWUeM', 'nEsgxvAq', 'NmAVTtfA', 'YTdCRVJt', 'QyBloWXZ', 'HKMQJANN', 'ZRrposmO',
                 'HfKRIwMb', 'UCAmikjV', 'uJYGhXqG', 'bxKGlBYX', 'nCzVgxgY', 'ltcNxFzI', 'MxOgekdE', 'JwtIxvKg',
                 'bEPKkJXP', 'cqUmYeAp', 'sFWbFEso', 'TqrXZaOw', 'galsfNtg', 'VIRwrkXp', 'gwhBRami', 'bPOwgKnT',
                 'fpHOwfAs', 'VXXLUaXP', 'btgWptTG', 'YWwNfVtR', 'bgoWYRMQ', 'bMudmjzJ', 'OMtioXZZ', 'bIBQTaHw',
                 'KcArMKAe', 'wwfmpuWA', 'znHDEHZP', 'kWFVfHWP', 'XwVALSPR', 'HHAeIHna', 'dCGNTMiG', 'ngwuvaCV',
                 'XSgHIFXD', 'ANBCxZzU', 'NanLCXEI', 'ZnBLVaqz', 'srPNUgVy', 'pCgBHqsR', 'wEbmsuJO', 'pWyRKfsb',
                 'udzhtHIr', 'IZFarbPw', 'lnfulcWk', 'QNLOXNwj', 'YFMZwKrU', 'RJQbcmKy', 'uizuNzbk', 'dlyiMEQt',
                 'TnWhKowI', 'LoYIbglA', 'GhJKwVWC', 'lVHmBCmb', 'qgxmqJKa', 'gfurxECf', 'hnrnuMte', 'LrQXqVUj',
                 'XDDOZFWf', 'QayGNSmS', 'ePtrWTFd', 'tbsBPHFD', 'naDKOzdk', 'xkUFKUoW', 'jVDpuAmP', 'SeZULMCT',
                 'AtGRGAYi', 'WTFJilSZ', 'NBfffJUe', 'mvgxfsRb', 'UXfyiodk', 'EftwspgZ', 'szowPwNq', 'BfGjiYom',
                 'iWEFJYkR', 'BCehjxAl', 'nqndbwXP', 'phwExnuQ', 'SzUcfjnr', 'PXtHzrqw', 'CNkSTLvx', 'tHFrzjai',
                 'MKozKLvT', 'pjHvJhoZ', 'zkbPtFyO', 'xZBEXWPR', 'dyGFeFAg', 'pKPTBZZq', 'bCYWWTxH', 'EQKKRGkR',
                 'cCsFudxF', 'muIetHMK', 'ishdUooQ', 'ItpCDLDM', 'ptEAnCSs', 'orfSPOJX', 'OKMtkqdQ', 'qTginJts',
                 'JzhdOhzb', 'THDtJuYh', 'jwEuQQve', 'rQAsGegu', 'kLkPtNnh', 'CtHqaXhY', 'FmSlImli', 'TiwRslOh',
                 'PWShFLnY', 'lFExzVaF', 'IKqsuNvV', 'CqqwKRSn', 'YUExUvhq', 'yaHLJxDD', 'qlZMvcWc', 'dqRtXzav',
                 'ktBqxSwa', 'GIMIxlmv', 'wKVwRQIp', 'UaXLYMMh', 'bKtkhUWD', 'HhKXJWno', 'tAYCAXge', 'aWlBVrkK',
                 'cDkXTaWP', 'hnmsRSvN', 'GHmAeUhZ', 'BIofZdtd', 'QZiSWCCB', 'CsGvKKBJ', 'OLpGAaEu', 'JCDeZBXq',
                 'HGPWuGlV', 'WuwrCsIY', 'AlDbXTlZ', 'hhold_size', 'ukWqmeSS', 'ukWqmeSS_max', 'ukWqmeSS_min', 
                 'mOlYV_ind_x', 'msICg_ind_x', 'YXCNt_ind_x', 'HgfUG_ind_x', 'EaHvf_ind_x', 'pdgUV_ind_x', 
                 'xrEKh_ind_x', 'QkRds_ind_x', 'XNPgB_ind_x', 'vvXmD_ind_x', 'KOjYm_ind_x', 'Qydia_ind_x', 
                 'vtkRP_ind_x', 'RPBUw_ind_x', 'QQdHS_ind_x', 'Hikoa_ind_x', 'SlRmt_ind_y', 'TRFeI_ind_y', 
                 'fmdsF_ind_y', 'lBMrM_ind_y', 'tMiQp_ind_y', 'wWIzo_ind_y', 'xnnDH_ind_y', 'CXizI_ind_y', 
                 'DQhEE_ind_y', 'LvUxT_ind_y', 'SSvEP_ind_y', 'YsahA_ind_y', 'lzzev_ind_y', 'ccbZA_ind_y', 
                 'fOUHD_ind_y', 'vkRKJ_ind_y', 'rwCRh_ind_y', 'yomtK_ind_y', 'iWGMu_ind_y', 'EaHvf_ind_y', 
                 'GmSKW_ind_y', 'tymHY_ind_y', 'yhUHu_ind_y', 'pdgUV_ind_y', 'qIbMY_ind_y', 'sDvAm_ind_y', 
                 'bszTA_ind_y', 'veBMo_ind_y', 'SowpV_ind_y', 'OeQKE_ind_y', 'XNPgB_ind_y', 'MxNAc_ind_y', 
                 'SuzRU_ind_y', 'PmhpI_ind_y', 'SjaWF_ind_y', 'TUafC_ind_y', 'bazjA_ind_y', 'dpMMl_ind_y', 
                 'qVwNL_ind_y', 'zTqjB_ind_y', 'BNylo_ind_y', 'CXjLj_ind_y', 'PwkMV_ind_y', 'Qydia_ind_y', 
                 'kVYrO_ind_y', 'VneGw_ind_y', 'rXEFU_ind_y', 'aKoLM_ind_y', 'SWhXf_ind_y', 'UCsCT_ind_y', 
                 'uJdwX_ind_y', 'qmOVd_ind_y', 'yOwsR_ind_y', 'ZIrnY_ind_y', 'dAmhs_ind_y', 'gCSRj_ind_y', 
                 'ESfgE_ind_y', 'okwnE_ind_y', 'OkXob_ind_y', 'dDnIb_ind_y', 'jVHyH_ind_y', 'xUYIC_ind_y']

mapped_Model_LightGBM_Level1_F09M01_clean = data_map.get_feature_mapping(Model_LightGBM_Level1_F09M01_clean)
pprint.pprint(mapped_Model_LightGBM_Level1_F09M01_clean, width=100, compact=True)




['cons_0305', 'farm_621', 'geo_district', 'cons_1336', 'gifts201', 'cons_1314', 'cons_0302',
 'farm_604', 'cons_1104', 'farm_615', 'hld_nbcellpho', 'cons_1404', 'hld_adeqcloth', 'cons_0801',
 'farm_623', 'cons_0602', 'cons_0912', 'cons_0101', 'cons_0504', 'cons_0202', 'cons_0824',
 'com_roadtype', 'cons_1417', 'cons_0403', 'cons_0603', 'hld_dwelloccu', 'cons_0413', 'cons_1405',
 'cons_0113', 'cons_0404', 'cons_0703', 'hld_rubbish', 'cons_0513', 'hld_credit1', 'cons_1103',
 'hld_busin9', 'cons_1304', 'cons_1204', 'hld_rooms', 'inc_102', 'cons_1303', 'cons_1324',
 'cons_0503', 'cons_0303', 'cons_1310', 'hld_adeqfood', 'own_528', 'cons_1308', 'hld_electricity',
 'cons_0606', 'cons_0802', 'own_507', 'farm_624', 'own_502', 'cons_0901', 'com_bank', 'cons_0915',
 'cons_0106', 'cons_0104', 'cons_1326', 'hld_selfscale', 'own_501', 'own_518', 'own_513',
 'cons_1214', 'cons_0109', 'cons_0111', 'cons_0204', 'cons_0501', 'cons_1322', 'cons_1101',
 'cons_0816', 'cons_1108', 'cons_1323', 'cons_1321',

In [11]:
Model_LightGBM_Level1_F08M01_clean = ['sDGibZrP', 'RMbjnrlm', 'GUvFHPNA', 'iwkvfFnL', 'goxNwvnG', 'HDMHzGif', 'MOIscfCf',
                    'tOWnWxYe', 'CtFxPQPT', 'fsXLyyco', 'ztGMreNV', 'YDgWYWcJ', 'pQmBvlkz', 'RLKqBexO', 
                    'BwkgSxCk', 'rfDBJtIz', 'cOSBrarW', 'lRGpWehf', 'dSALvhyd', 'WbxAxHul', 'NitzgUzY', 
                    'bhFgAObo', 'mnIQKNOM', 'GYTJWlaF', 'lTAXSTys', 'IBPMYJlv', 'WbEDLWBH', 'cgJgOfCA', 
                    'hTraVEWP', 'nKoaotpH', 'OnTaJkLa', 'EMDSHIlJ', 'NGOnRdqc', 'vmZttwFZ', 'tjrOpVkX', 
                    'zXPyHBkn', 'dkoIJCbY', 'hJrMTBVd', 'xNUUjCIL', 'rnJOTwVD', 'dAaIakDk', 'WqhniYIc', 
                    'HfOrdgBo', 'wBXbHZmp', 'FGYOIJbC', 'CbzSWtkF', 'TzPdCEPV', 'lybuQXPm', 'GDUPaBQs',
                    'EfkPrfXa', 'JeydMEpC', 'jxSUvflR', 'VFTkSOrq', 'CpqWSQcW', 'iVscWZyL', 'JMNvdasy', 
                    'NrvxpdMQ', 'nGMEgWyl', 'pyBSpOoN', 'zvjiUrCR', 'aCfsveTu', 'TvShZEBA', 'TJUYOoXU', 
                    'sYIButva', 'cWNZCMRB', 'yeHQSlwg', 'nSzbETYS', 'CVCsOVew', 'UXSJUVwD', 'FcekeISI', 
                    'QBJeqwPF', 'mBlWbDmc', 'MBQcYnjc', 'KHzKOKPw', 'LrDrWRjC', 'TFrimNtw', 'InULRrrv', 
                    'fhKiXuMY', 'fxbqfEWb', 'GnUDarun', 'XVwajTfe', 'yHbEDILT', 'JbjHTYUM', 'mHzqKSuN',
                    'ncjMNgfp', 'dkPWxwSF', 'dsIjcEFe', 'ySkAFOzx', 'QzqNzAJE', 'bgfNZfcj', 'tZKoAqgl', 
                    'NrUWfvEq', 'SsZAZLma', 'mNrEOmgq', 'hESBInAl', 'ofhkZaYa', 'mDTcQhdH', 'mvGdZZcs', 
                    'ALbGNqKm', 'wgWdGBOp', 'nuwxPLMe', 'vRIvQXtC', 'rAkSnhJF', 'rtPrBBPl', 'tMJrvvut', 
                    'BbKZUYsB', 'LjvKYNON', 'uZGqTQUP', 'NIRMacrk', 'UBanubTh', 'dEpQghsA', 'WiwmbjGW', 
                    'ULMvnWcn', 'AsEmHUzj', 'BMmgMRvd', 'QqoiIXtI', 'duayPuvk', 'YKwvJgoP', 'ytYMzOlW',
                    'YXkrVgqt', 'sslNoPlw', 'IIEHQNUc', 'ErggjCIN', 'tlxXCDiW', 'eeYoszDM', 'KAJOWiiw', 
                    'UCnazcxd', 'uVnApIlJ', 'ZzUrQSMj', 'nGTepfos', 'ogHwwdzc', 'eoNxXdlZ', 'kZVpcgJL', 
                    'lFcfBRGd', 'UXhTXbuS', 'UsENDgsH', 'wxDnGIwN', 'rYvVKPAF', 'OybQOufM', 'wnESwOiN', 
                    'glEjrMIg', 'iBQXwnGC', 'VBjVVDwp', 'lOujHrCk', 'wakWLjkG', 'RJFKdmYJ', 'ZmJZXnoA', 
                    'lQQeVmCa', 'ihGjxdDj', 'mycoyYwl', 'FlBqizNL', 'CIGUXrRQ', 'YlZCqMNw', 'gllMXToa',
                    'DbUNVFwv', 'EuJrVjyG', 'uRFXnNKV', 'gfmfEyjQ', 'ggNglVqE']    

mapped_Model_LightGBM_Level1_F08M01_clean = data_map.get_feature_mapping(Model_LightGBM_Level1_F08M01_clean)
pprint.pprint(mapped_Model_LightGBM_Level1_F08M01_clean, width=100, compact=True)




['cons_0209', 'cons_1339', 'cons_0115', 'cons_0116', 'own_526', 'own_515', 'cons_0708', 'cons_0709',
 'cons_0117', 'cons_1415', 'cons_1407', 'own_519', 'cons_0804', 'cons_0206', 'cons_0815',
 'farm_611', 'cons_1337', 'cons_1412', 'cons_1338', 'cons_0309', 'cons_0706', 'cons_0823',
 'hld_busin6', 'cons_0813', 'cons_1210', 'own_506', 'own_520', 'cons_1335', 'cons_0830',
 'cons_1216', 'cons_0114', 'cons_0903', 'cons_0412', 'cons_0704', 'hld_mtltel', 'cons_0609',
 'cons_0910', 'cons_1334', 'cons_0818', 'cons_0707', 'cons_0909', 'cons_0514', 'cons_1406',
 'cons_1212', 'own_532', 'cons_0902', 'cons_0310', 'hld_busin8', 'cons_0110', 'own_505',
 'cons_0208', 'own_530', 'own_531', 'own_514', 'cons_1218', 'farm_614', 'cons_0509', 'cons_0817',
 'cons_0515', 'cons_1333', 'cons_0610', 'cons_0814', 'cons_0604', 'cons_0810', 'cons_1414',
 'hld_credit2', 'inc_114', 'cons_0705', 'hld_whynoelec', 'farm_618', 'cons_0828', 'cons_1312',
 'cons_0107', 'cons_0811', 'cons_1106', 'cons_1306', 'cons_1410', 'con

In [12]:
data_map.get_feature_mapping(['CaukPfUC', 'MUrHEJeh', 'MzEtIdUF', 'XizJGmbu', 'rQWIpTiG'])

['ind_rwenglish', 'ind_work6', 'ind_health5', 'ind_educ01', 'ind_work2']